
<b><font face="微软雅黑" size="5" color="lightblue">Transformer网络详解</font></b>

<b><font face="微软雅黑" size="4" color="lightblue">导入相关库</font></b>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import matplotlib.pyplot as plt

<b><font face="微软雅黑" size="4" color="lightblue">Transformer - Encoder</font></b>

Encoder输入：文本的token通过Word Embedding映射为固定维度的词向量，一般为512维，再将词向量序列和位置编码相加变为多头注意力网络的输入。

PositionalEncoding:

<img src="Encoder.png" width="480" height="400" alt="Encoder">

In [ ]:
#定义参数
embed_size = 512  #词嵌入维度, 每个token经过Embeding后的维度
head_nums = 8     #多头注意力头数
head_size = embed_size // head_nums #每个头的输入维度
hidden_size = 512 #每个头的输出维度

#单个注意力代码实现
#*args：接收任意个数的位置参数，打包成元组 (tuple) 传入函数内部；
#**kwargs：接收任意个数的关键字参数，打包成字典 (dict) 传入函数内部；
class SingleAttention(nn.Module):
    #初始化
    def __init__(self, *args, **kwargs):
        #初始化父类
        super.__init__(*args,**kwargs)
        #定义Q,K,V的线性变换层
        self.Q_linear = nn.Linear(head_size,hidden_size)
        self.K_linear = nn.Linear(head_size,hidden_size)
        self.V_linear = nn.Linear(head_size,hidden_size)

    def forward(self,x):
        batch_size, seq_len, head_size = x.size()
        #获取Q,K,V
        Q = self.Q_linear(x)   #查询矩阵维度：(batch_size, seq_len, hidden_size)
        K = self.K_linear(x)   #键矩阵维度：(batch_size, seq_len, hidden_size) 
        V = self.V_linear(x)   #值矩阵维度：(batch_size, seq_len, hidden_size)

        #计算注意力分数
        #注意力分数维度：(batch_size, seq_len, seq_len) 每个位置与其他位置的相关性
        scores = Q @ K.transpose(-2, -1) / math.sqrt(head_size)  

        #计算注意力权重, 在最后一维上进行softmax归一化，得到每个位置对当前token的权重，且所有权重和为1
        attention_weights = F.softmax(scores, dim=-1)

        #计算加权和
        output = torch.matmul(attention_weights, V)

        return output, attention_weights

        


单头注意力计算：输入的数据维度为 batch_size, head_nums, seq_length, 词向量维度/head_nums;输入数据分别乘wq，wk，wv 得到Q,K,V，Q,K矩阵点乘输出不同位置词向量和其余位置词向量之间的余弦相似度，再通过Scaling（结果除根号dk,dk是单个头输入的词向量维度）操作防止乘积过大而导致softmax归一化后梯度极小的问题，最后将softmaxs输出的结果和V矩阵相乘。

In [ ]:
embed_size = 512
head_nums = 12
head_size = embed_size // head_nums
hidden_size = 512
dropout = 0.1

# 单个注意力
class Attention(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # 计算Q，K，V
        self.query = nn.Linear(head_size, hidden_size)
        self.key = nn.Linear(head_size, hidden_size)
        self.value = nn.Linear(head_size, hidden_size)

        # 创建mask
        self.register_buffer("attention_mask", torch.tril(torch.ones(head_size, head_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        batch_size, seq_len, head_size = x.size()
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)
        weight = q @ k.transpose(-2,-1)
        weight = weight.masked_fill(self.attention_mask[:seq_len,:seq_len] == 0, float('-inf')) / math.sqrt(head_size)
        weight = F.softmax(weight, -1)
        weight = self.dropout(weight)
        output = weight @ v
        return  output

多头注意力计算：多头注意力即多个单头的叠加，针对一个词向量来说，假如一个词向量的维度是512，头数为8，那么每个头的输入就是64维，每个头会将词向量的不同部分作为输入计算，关注词向量不同维度下的特征，最后将所有头的输出拼接。多头注意力输出后会经过残差和层归一化操作。

<b><font face="微软雅黑" size="4" color="lightblue">Transformer - Decoder</font></b>